<img src="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png" srcset="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_130 130w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_260 260w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_390 390w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_520 520w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_650 650w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_780 780w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_910 910w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1040 1040w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1170 1170w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1290 1290w" sizes="100vw" width="1290">
<p style='margin-top: 1rem; margin-bottom: 1rem;'>Developed by Marco Frodl, Principal Consultant for Generative AI @ <a href='https://go.mfr.one/tt-en' _target='blank'>Thinktecture AG</a> -- More about me on my <a href='https://go.mfr.one/marcofrodl-en' _target='blank'>profile page</a></p>

## Prettify Colab Notebook outputs

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

### Set API keys via Colab Secrets

In [ ]:
# import Colab Secrets userdata module
from google.colab import userdata

# set OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# set Langfuse API keys
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('LANGFUSE_PUBLIC_KEY')
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get('LANGFUSE_SECRET_KEY')

## Load libraries

In [ ]:
!pip -q install protobuf==3.20.3 langchain==0.3.1 langchain-openai==0.2.1 langchain-mistralai==0.2.0 langfuse==2.51.2

In [ ]:
# let's check the imported versions of the most important libraries
!pip show langchain
!pip show langchain-openai
!pip show langchain-mistralai
!pip show langfuse

## Prepare Langfuse

In [ ]:
# prepare Langfuse as debugging and tracing framework for our Generative AI application - never develop GenAI apps without that!
from langfuse.callback import CallbackHandler
handler = CallbackHandler()

## Prepare LLM

### OpenAI LLM

In [ ]:
from langchain_openai import ChatOpenAI
llm_openai = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=1000) # gpt-4o-mini, gpt-4o

### Mistral AI LLM

In [ ]:
from langchain_mistralai import ChatMistralAI
llm_mistralai = ChatMistralAI(mistral_api_key=userdata.get('MISTRAL_API_KEY'), temperature=0, max_tokens=1000, model="open-mistral-nemo-2407") # open-mistral-nemo-2407, mistral-large-2407, mistral-small-2409

## Simple LLM chat call

In [ ]:
# pure as water
result = llm_openai.invoke("Which is the best generative ai large language model in the world?")
print("--- OpenAI ---")
print(result.content)
print(result.response_metadata)

result = llm_mistralai.invoke("Which is the best generative ai large language model in the world?")
print("--- MistralAI ---")
print(result.content)
print(result.response_metadata)


## Chat with Metadata

In [ ]:
# little bit more context about who is talking
from langchain_core.messages import HumanMessage
result = llm_openai.invoke([HumanMessage(content="Kennst du meinen Namen?", name="Marco")])
print("--- OpenAI ---")
print(result.content)
print(result.response_metadata)

result = llm_mistralai.invoke([HumanMessage(content="Kennst du meinen Namen?", name="Marco")]) # name is not supported by ChatMistralAI
print("--- MistralAI ---")
print(result.content)
print(result.response_metadata)

## Chat with Tracing

In [ ]:
# lets optimize and trace whats happening
from langchain_core.messages import HumanMessage, SystemMessage
messages = [SystemMessage(content="Du bist eine AI, die den User Duzt und gerne Wörter wie Bro, Digga und ich schwör verwendet. Der Name des Users ist Marco")]
messages.extend([HumanMessage(content="Kennst du meinen Namen?")])

result = llm_openai.invoke(messages,{"callbacks":[handler]})
print("--- OpenAI ---")
print(result.content)
print(result.response_metadata)

result = llm_mistralai.invoke(messages,{"callbacks":[handler]})
print("--- MistralAI ---")
print(result.content)
print(result.response_metadata)

## Chat with history

In [ ]:
# Dialog based chat
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
messages = [
    SystemMessage(content="Du bist eine AI, die sich mit Popmusik auskennt und viele Fakten in Antworten einfliessen lässt. Du Duzt den User."),
    HumanMessage(content="Genesis war ohne Peter Gabriel ein Schatten seiner selbt. Wer war danach der Sänger?"),
    AIMessage(content="Phil Collins - allerdings muss ich beim Thema Erfolg widersprechen!"),
    HumanMessage(content="Ok, warum?")]

result = llm_openai.invoke(messages,{"callbacks":[handler]})
print("--- OpenAI ---")
print(result.content)
print(result.response_metadata)

result = llm_mistralai.invoke(messages,{"callbacks":[handler]})
print("--- MistralAI ---")
print(result.content)
print(result.response_metadata)